# Lineaarinen ja logistinen regressio

## Tavoitteet
Tavoitteena on ennustaa talon hintaa per neliömetri New Taipei City, Taiwanissa. Käytämme mallinnuksessa lineaarista ja logistista regressiota. Tutkimme datasetistä löytyviä talojen arvoja ja käytämmä näitä koneoppimismallien kouluttamiseen ja testaamiseen.


In [5]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
real_estate_valuation = fetch_ucirepo(id=477)
# data (as pandas dataframes)
X = real_estate_valuation.data.features
y = real_estate_valuation.data.targets
# metadata
print(real_estate_valuation.metadata)
# variable information
print(real_estate_valuation.variables)

{'uci_id': 477, 'name': 'Real Estate Valuation', 'repository_url': 'https://archive.ics.uci.edu/dataset/477/real+estate+valuation+data+set', 'data_url': 'https://archive.ics.uci.edu/static/public/477/data.csv', 'abstract': 'The real estate valuation is a regression problem. The market historical data set of real estate valuation are collected from Sindian Dist., New Taipei City, Taiwan. ', 'area': 'Business', 'tasks': ['Regression'], 'characteristics': ['Multivariate'], 'num_instances': 414, 'num_features': 6, 'feature_types': ['Integer', 'Real'], 'demographics': [], 'target_col': ['Y house price of unit area'], 'index_col': ['No'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2018, 'last_updated': 'Mon Feb 26 2024', 'dataset_doi': '10.24432/C5J30W', 'creators': ['I-Cheng Yeh'], 'intro_paper': {'title': 'Building real estate valuation models with comparative approach through case-based reasoning', 'authors': 'I. Yeh, Tzu-Kuang Hsu', 'published_

In [9]:
X.describe(include="all")

,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude
count,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000
mean,2013.148971,17.712560,1083.885689,4.094203,24.969030,121.533361
std,0.281967,11.392485,1262.109595,2.945562,0.012410,0.015347
min,2012.667000,0.000000,23.382840,0.000000,24.932070,121.473530
25%,2012.917000,9.025000,289.324800,1.000000,24.963000,121.528085
50%,2013.167000,16.100000,492.231300,4.000000,24.971100,121.538630
75%,2013.417000,28.150000,1454.279000,6.000000,24.977455,121.543305
max,2013.583000,43.800000,6488.021000,10.000000,25.014590,121.566270


## Datan kuvaus
Datasetti pitää sisällään erilaisia tietoja taloista Taiwanissa. Datasetti sisältää seuraavat sarakkeet:
- X1 transaction date: Talon myynnin päivämäärä
- X2 house age: Talon ikä
- X3 distance to the nearest MRT station: Etäisyys lähimpään metroasemaan
- X4 number of convenience stores: Lähellä olevien ruokakauppojen määrä
- X5 latitude: Talon leveysaste
- X6 longitude: Talon pituusaste

Näillä arvoilla tavoitteenamme on ennustaa talon hintaa per neliömetri.


## Datan esikäsittely

## Mallinnus

In [ ]:
# linear regression

In [10]:
# logistic regression

## Arviointi

## Käyttöönotto
Ennustukset voisivat olla hyödyllisiä esimerkiksi kiinteistövälittäjille tai ihmisille, jotka haluavat myydä omia kiinteistöjä. Datasetti perustuu juuri New Taipei Cityyn, joten ennustukset pätevät tarkasti ainoastaan tähän kaupunkiin. Ennustuksille tarvittaisiin luultavasti oma sovellus tai nettisivu, joka tarjoisi käyttäjälle helpon käyttäjäkokemuksen. Jos sovellusta haluttaisiin laajentaa, tarvittaisiin lisää dataa eri kaupungeille.